In [ ]:

    # scoring.py
# script to tie everything together and determine score of thrown dart


# import scripts
from camera import *

# import libraries
import pyfirmata
import paho.mqtt.publish as publish
import atexit

def release_cameras(camX, camY):
    try:
        camX.cam.release()
        camY.cam.release()
        print("Cameras released")
    except:
        print("Cameras unable to be released")
    return

def initalize_cameras(image_path):
    # initialize cameras
    try:
        camY = Camera(0, image_path)
        camX = Camera(2, image_path)
        # release cameras when program exits
        atexit.register(release_cameras, camX, camY)
        return  camX, camY
    except:
        print("Gotta reboot or something to release those camera objects :/")
        exit()

def display_image(img):
    image_height, image_width, channels = img.shape
    # # use cv2 to display image with red vertical line through center of image
    # cv2.line(img, (int(image_width/2), 0), (int(image_width/2), image_height), (0, 0, 255), 2)
    # cv2.imshow('image', img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    plt.imshow(img)
    plt.plot([int(image_width/2), int(image_width/2)], [0, image_height], 'r-')
    plt.show()
    return


# set directory paths
cur_dir = os.getcwd()
parent_dir = os.path.dirname(cur_dir)
image_path = cur_dir + "/images/test"
print("current dir: ", cur_dir)
print("parent dir: ", parent_dir)
print("image path: ", image_path)   

In [ ]:
camX, camY = initalize_cameras(image_path)

image_nodartX, image_nodartY, image_dartX, image_dartY = camX.load_images()

display_image(image_nodartX)
display_image(image_nodartY)
display_image(image_dartX)
display_image(image_dartY)

 # capture time it takes to release cameras
start_time = time.time()
release_cameras(camX, camY)
end_time = time.time()
print("Time to release cameras: ", end_time - start_time)

In [ ]:
# get dart tip coordinates from images
image_coordinates = camX.get_image_coordinates()
print("image coordinates: ", image_coordinates)


# calculate angles of dart to cameras
angles_to_dart = camX.compute_angles(image_coordinates)
print("angles to dart: ", angles_to_dart)


# convert coordinates to be relative to dart board
#cam_dist = 381 #mm
x1, y1 = 445, 0
x2, y2 = 0, 430
board_radius = 228.6
x_dart, y_dart = camX.get_board_coordinates(angles_to_dart, x1, y1, x2, y2)
print("board coordinates: ", x_dart, y_dart)


# calculate score of throw and send to app
score, channel, slice_area = camX.get_score(x_dart, y_dart)
publish.single(str(channel), str(slice_area), hostname=MQTT_SERVER, auth=authentications)


# Create Visualiation of Dart Board
fig, ax = plt.subplots() 
ax.set_xlim(-400, 400)
ax.set_ylim(-400, 400)
dart_board = plt.Circle((0, 0), board_radius, edgecolor='black', fill=False)
#dart_board = ax.pie([.05]*20)
dart = plt.Arrow(x_dart, y_dart, 15, 15, color='black')
bullseye = plt.Circle((0, 0), 10, edgecolor='black', fill=False)
camA = plt.Rectangle((x1, y1), 10, 10, edgecolor='black', fill=False, clip_on=False)
camB = plt.Rectangle((x2, y2), 10, 10, edgecolor='black', fill=False, clip_on=False) 
ax.add_patch(dart_board)
ax.add_patch(dart)
ax.add_patch(bullseye)
ax.add_patch(camA)
ax.add_patch(camB)
leg_Cam1_Dart = ax.axline((x1, y1), slope = camX.slope1)
leg_Cam2_Dart = ax.axline((x2, y2), slope = camX.slope2)
ax.plot(x_dart, y_dart)
plt.show()

option = input("1 to run program again, 0 to exit: ")
